In [1]:
import os, json

In [21]:
def perfect_match(v1,v2,k):
    if(len(v1)!=len(v2)):
        return 0
    delta = abs(v1[0] - v2[0])
    for i in range(len(v1)):
        if(abs(v1[i]-v2[i])-delta > k):
            #print(delta, v1[i],v2[i])
            return 0
    return 1

def perfect_align_clustering(phrases_vec,k):
    mp = {}
    remap = {}
    id = 0
    phrases = []
    for phrase, vec in phrases_vec.items():
        phrases.append(phrase)

    for i in range(len(phrases)):
        pi = phrases[i]
        vi = phrases_vec[pi]
        if(pi not in mp):
            mp[pi] = id
            remap[id] = [pi]
            id += 1
        else:
            continue
        for j in range(i+1, len(phrases)):
            pj = phrases[j]
            vj = phrases_vec[pj]

            #optimization: pj in mp denotes that pj must be at least perfectly match with one phrase before pi, but pi is not in mp
            #so pi and pj must be not perfectly aligned, and thus skip comparison 
            if(pj in mp):
                continue

            if(perfect_match(vi,vj,k) == 1):
                mp[pj] = id-1
                remap[id-1].append(pj)
    
    return mp, remap

In [15]:


def read_dict(file):
    print(file)
    with open(file, 'r') as file:
        data = json.load(file)
    return data 

def read_file(file):
    data = []
    with open(file, 'r') as file:
        # Iterate over each line in the file
        for line in file:
            # Print the line (you can replace this with other processing logic)
            data.append(line.strip())
    return data

def get_extracted_path(path):
    path = path.replace('raw','extracted')
    path = path.replace('.pdf','.txt')
    return path

def get_relative_location_path(extracted_path):
    path = extracted_path[:-4] + '_relative_location.csv'
    return path

def get_root_path():
    current_path = os.path.abspath(os.path.dirname(__file__))
    parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
    #print("Parent path:", parent_path)
    return parent_path

def get_key_path(extracted_path):
    path = extracted_path.replace('extracted','truths/key_truth')
    return path

In [22]:
root_path = '/Users/yiminglin/Documents/Codebase/Pdf_reverse'
tested_paths = []
tested_paths.append(root_path + '/data/raw/complaints & use of force/Champaign IL Police Complaints/Investigations_Redacted.pdf')
tested_paths.append(root_path + '/data/raw/complaints & use of force/UIUC PD Use of Force/22-274.releasable.pdf')
tested_paths.append(root_path + '/data/raw/certification/CT/DecertifiedOfficersRev_9622 Emilie Munson.pdf')
tested_paths.append(root_path + '/data/raw/certification/IA/Active_Employment.pdf')
tested_paths.append(root_path + '/data/raw/certification/MT/RptEmpRstrDetail Active.pdf')
tested_paths.append(root_path + '/data/raw/certification/VT/Invisible Institue Report.pdf')

tested_id = 1 #starting from 0

path = tested_paths[tested_id]
extracted_path = get_extracted_path(path)
out_path = get_relative_location_path(extracted_path)
phrases = read_dict(out_path)
mp, remap = perfect_align_clustering(phrases,1)

for k,v in remap.items():
    if(len(v) > 1):
        print(v)

/Users/yiminglin/Documents/Codebase/Pdf_reverse/data/extracted/complaints & use of force/UIUC PD Use of Force/22-274.releasable_relative_location.csv
['Report Criteria', 'Incidents Between', '1/1/2021 AND 12/31/2021', '21-00017', '1143', 'Ahart, Rachael F.', 'Apartment', 'Search Warrant', 'Entered on 4/6/2021 9:55:14 AM by jbclark.', 'Harvey, Jamir T.', 'Officers from the Street Crimes Task Force executed a search warrant and subject fled from scene.', 'Officers pursued on foot and Officer Snow located subject in a', 'parking lot.', 'Officer Snow pointed firearm at the suspect and directed him to get on the ground and suspect complied.', 'Suspect was taken into custody without further', '3/14/2022 8:27:19 AM', '21-01236', '1/31/2021', '0156', 'Benoit, Robert W.', '2/2/2021', 'Green & Second Streets', 'Roadway', 'Armed Subject', 'Entered on 4/6/2021 10:04:34 AM by jbclark.', 'Norman, Shannon L', '1114 1/2 Ashford Ct', 'Mahomet', 'Active Aggressive', 'Officers responded to 35 E. Green to

In [24]:
print(phrases['Officer'])
print(phrases['Case Number'])

[90, 204, 318, 334, 350, 366, 478, 494, 510, 655, 671, 687, 703, 814, 924]
[6, 120, 232, 397, 543, 733, 843]


In [20]:
s1 = [6, 120, 232, 397, 543, 733, 843]
s2 = [18, 132, 244, 409, 554, 745, 855]

perfect_match(s1,s2)

12 543 554


0

## Observation 
- Perfect aligned is too strict. Relax it to be the delta distance. 